In [1]:
import os
from multiprocessing import cpu_count

import numpy as np
import tensorflow as tf

from itertools import islice, cycle, tee, chain

from model_wrangler.model.text_tools import TextProcessor

DATA_DIR = '/Users/mcmenamin/GitHub/sundries/moby_sequel/data'
WORK_DIR = '/Users/mcmenamin/GitHub/sundries/moby_sequel/tf/'
TEXT_PROCESSOR = TextProcessor()

train_file = os.path.join(DATA_DIR, 'train_data.txt')
test_file = os.path.join(DATA_DIR, 'test_data.txt')

model_dir = os.path.join(WORK_DIR, "keras_model")
os.makedirs(model_dir, exist_ok=True)

In [2]:
def consume(iterator, n_steps):
    """Advance an iterator n_steps ahead. If n is none, consume entirely."""
    if n_steps is None:
        deque(iterator, maxlen=0)
    else:
        next(islice(iterator, n_steps, n_steps), None)

def sliding_window(iterable, win_len=2):
    "Return samples from a sliding window of length win_len"

    if isinstance(iterable, np.ndarray):
        iterable = iterable.ravel()

    iters = tee(iterable, win_len)
    for i, it in enumerate(iters):
        consume(it, i)
    return zip(*iters)


def text_input_fn(filename, window_len=120, num_epochs=1, batch_size=64):

    def data_gen():
        with open(filename, 'rt') as file:
            for line in file:
                line_int = TEXT_PROCESSOR.string_to_ints(line)
                for window in sliding_window(line_int, window_len + 1):
                    win_0 = np.zeros((window_len, TEXT_PROCESSOR.num_chars + 2))
                    win_1 = np.zeros(TEXT_PROCESSOR.num_chars + 2)

                    for row, col in enumerate(window[:-1]):
                        win_0[row, col] = 1.0
                    win_1[window[-1]] = 1
                    yield win_0, win_1

    dataset = (
        tf.data.Dataset.
        from_generator(
            data_gen,
            (tf.float32, tf.float32),
            (tf.TensorShape([window_len, TEXT_PROCESSOR.num_chars + 2]),
             tf.TensorShape([TEXT_PROCESSOR.num_chars + 2])
            )
        ).
        shuffle(buffer_size=256).
        repeat(num_epochs).
        batch(batch_size).
        make_one_shot_iterator()
    )

    in_tensor, targ_tensor = dataset.get_next()
    return {'lstm0_input': in_tensor}, targ_tensor


def sample_char(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64').ravel()

    probs = np.exp(preds - temperature)
    probs /= np.sum(probs)

    char = np.random.choice(
        list(TEXT_PROCESSOR.int_to_char.values()),
        p=probs
    )
    return char

def text_pred_fn(text, window_len=120, num_epochs=1, batch_size=64):

    def data_gen():
        text_int = TEXT_PROCESSOR.string_to_ints(text)[-(window_len + 1):]
        win_0 = np.zeros((window_len, TEXT_PROCESSOR.num_chars + 2))
        win_1 = np.zeros(TEXT_PROCESSOR.num_chars + 2)

        for row, col in enumerate(text_int[:-1]):
            win_0[row, col] = 1.0
        win_1[text_int[-1]] = 1
        yield win_0, win_1


    dataset = (
        tf.data.Dataset.
        from_generator(
            data_gen,
            (tf.float32, tf.float32),
            (tf.TensorShape([window_len, TEXT_PROCESSOR.num_chars + 2]),
             tf.TensorShape([TEXT_PROCESSOR.num_chars + 2])
            )
        ).
        batch(1).
        make_one_shot_iterator()
    )

    in_tensor, targ_tensor = dataset.get_next()
    return {'lstm0_input': in_tensor}, targ_tensor


In [3]:
def set_max_threads(max_threads=None):
    sess_cfg = tf.ConfigProto()

    if max_threads is None:
        max_threads = cpu_count()

    sess_cfg.intra_op_parallelism_threads = max_threads
    sess_cfg.inter_op_parallelism_threads = max_threads
    sess_cfg.allow_soft_placement = True
    return sess_cfg

In [ ]:
model.add(
    tf.keras.layers.LSTM(
        128,
        activation='tanh',
        return_sequences=True,
        input_shape=(win_len, TEXT_PROCESSOR.num_chars + 2),
        unroll=True,
        dropout=0.2,
        name='lstm0'
    )
)

model.add(
    tf.keras.layers.LSTM(
        128,
        activation='tanh',
        return_sequences=False,
        input_shape=(win_len, 128),
        unroll=True,
        name='lstm1'
    )
)

In [4]:
win_len = 80

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():

model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.LSTM(
        4,
        activation='tanh',
        return_sequences=False,
        input_shape=(win_len, TEXT_PROCESSOR.num_chars + 2),
        unroll=True,
        name='lstm0'
    )
)

model.add(
    tf.keras.layers.Dense(
        TEXT_PROCESSOR.num_chars + 2,
        activation='softmax',
        name='dense'
    )
)

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(lr=0.01)
)

train_spec = tf.estimator.TrainSpec(
    lambda: text_input_fn(train_file, window_len=win_len, num_epochs=3, batch_size=64),
    max_steps=80000
)

eval_spec = tf.estimator.EvalSpec(
    lambda: text_input_fn(test_file, window_len=win_len, num_epochs=1, batch_size=64),
    steps=500
)


estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir=model_dir,
    config=tf.estimator.RunConfig(
        session_config=set_max_threads()
    )
)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Using the Keras model from memory.
INFO:tensorflow:Using config: {'_task_id': 0, '_tf_random_seed': None, '_task_type': 'worker', '_master': '', '_is_chief': True, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_model_dir': '/Users/mcmenamin/GitHub/sundries/moby_sequel/tf/keras_model', '_keep_checkpoint_every_n_hours': 10000, '_session_config': intra_op_parallelism_threads: 4
inter_op_parallelism_threads: 4
allow_soft_placement: true
, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x123fce780>, '_service': None}


In [5]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /Users/mcmenamin/GitHub/sundries/moby_sequel/tf/keras_model/keras_model.ckpt
INFO:tensorflow:Saving checkpoints for 1 into /Users/mcmenamin/GitHub/sundries/moby_sequel/tf/keras_model/model.ckpt.
INFO:tensorflow:loss = 4.157319, step = 1
INFO:tensorflow:global_step/sec: 6.14164
INFO:tensorflow:loss = 3.1655025, step = 101 (16.283 sec)
INFO:tensorflow:global_step/sec: 18.6484
INFO:tensorflow:loss = 2.9805465, step = 201 (5.362 sec)
INFO:tensorflow:global_step/sec: 18.1591
INFO:tensorflow:loss = 2.8036602, step = 301 (5.508 sec)
INFO:tensorflow:global_step/sec: 18.7322
INFO:tensorflow:loss = 2.8294916, step = 401 (5.338 sec)
INFO:tensorflow:global_step/sec: 17.875
INFO:tensorflow:loss = 2.618784

INFO:tensorflow:global_step/sec: 19.5115
INFO:tensorflow:loss = 2.1053975, step = 7901 (5.126 sec)
INFO:tensorflow:global_step/sec: 19.3629
INFO:tensorflow:loss = 2.184816, step = 8001 (5.164 sec)
INFO:tensorflow:global_step/sec: 19.5647
INFO:tensorflow:loss = 2.0749483, step = 8101 (5.111 sec)
INFO:tensorflow:global_step/sec: 19.4757
INFO:tensorflow:loss = 2.7653084, step = 8201 (5.135 sec)
INFO:tensorflow:global_step/sec: 19.4983
INFO:tensorflow:loss = 2.2317936, step = 8301 (5.129 sec)
INFO:tensorflow:global_step/sec: 19.622
INFO:tensorflow:loss = 2.192801, step = 8401 (5.096 sec)
INFO:tensorflow:global_step/sec: 19.5117
INFO:tensorflow:loss = 2.433471, step = 8501 (5.125 sec)
INFO:tensorflow:global_step/sec: 19.6161
INFO:tensorflow:loss = 2.6430542, step = 8601 (5.098 sec)
INFO:tensorflow:global_step/sec: 19.7307
INFO:tensorflow:loss = 2.1928043, step = 8701 (5.068 sec)
INFO:tensorflow:global_step/sec: 19.187
INFO:tensorflow:loss = 2.280572, step = 8801 (5.212 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 19.4889
INFO:tensorflow:loss = 3.8167405, step = 15002 (5.131 sec)
INFO:tensorflow:global_step/sec: 19.3477
INFO:tensorflow:loss = 3.1677763, step = 15102 (5.168 sec)
INFO:tensorflow:global_step/sec: 19.6026
INFO:tensorflow:loss = 2.1574316, step = 15202 (5.101 sec)
INFO:tensorflow:global_step/sec: 19.5795
INFO:tensorflow:loss = 2.032277, step = 15302 (5.108 sec)
INFO:tensorflow:global_step/sec: 17.4828
INFO:tensorflow:loss = 2.205441, step = 15402 (5.721 sec)


KeyboardInterrupt: 

In [ ]:
new_text = 'You can still call me Ishmael. '
for _ in range(100):
    prob_dist = next(estimator.predict(lambda: text_pred_fn(new_text)))['dense']
    next_char = sample_char(prob_dist)
    new_text += next_char
print(new_text)

In [7]:
print(new_text)

You can still call me Ishmael. ?TEWn5OR1? NQFdnFVu?


In [ ]:
_